# Programowanie HTTP w Pythonie

Od prostych zapytań po asynchroniczne klienty i mini API.


## Cele
- opanować podstawy pracy z biblioteką `requests`
- poznać asynchroniczne podejście (`aiohttp`)
- zbudować szkic API z FastAPI / Flask


## Klient synchroniczny `requests`
Najprostsza droga do wysyłania zapytań HTTP.


In [ ]:
import requests

response = requests.get("https://httpbin.org/get", params={"q": "python"})
print(response.status_code)
print(response.json()["args"])


## Klient asynchroniczny `aiohttp`
Asynchroniczność obsługuje wiele zapytań jednocześnie bez blokowania.


In [ ]:
import asyncio
import aiohttp

async def fetch(session, url):
    async with session.get(url) as resp:
        data = await resp.json()
        return url, data["url"]

async def main():
    urls = ["https://httpbin.org/get" for _ in range(3)]
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        for result in await asyncio.gather(*tasks):
            print(result)

asyncio.run(main())


## Mini API z FastAPI
FastAPI pozwala szybko tworzyć RESTowe endpoints z walidacją Pydantic.


In [ ]:
api_example = """
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float

@app.post("/items")
def create_item(item: Item):
    return {"status": "ok", "item": item}

# uruchom: uvicorn main:app --reload
"""
print(api_example)


**Podsumowanie:** HTTP w Pythonie obejmuje synchroniczne i asynchroniczne biblioteki oraz lekkie frameworki.

**Pytanie kontrolne:** Dlaczego `asyncio` lepiej skaluje się przy wielu zapytaniach?


### 🧩 Zadanie 1
Napisz funkcję, która pobiera listę URL-i przy użyciu `requests` i zwraca słownik `url -> status`.


In [ ]:
# Rozwiązanie Zadania 1
import requests

def fetch_status(urls):
    result = {}
    for url in urls:
        try:
            resp = requests.head(url, timeout=3)
            result[url] = resp.status_code
        except requests.RequestException as exc:
            result[url] = str(exc)
    return result

print(fetch_status(["https://python.org", "https://httpbin.org/status/404"]))


### 🧩 Zadanie 2
Przygotuj asynchroniczną funkcję, która pobierze dane JSON z wielu endpointów i połączy je w jedną listę rekordów.


In [ ]:
# Rozwiązanie Zadania 2
import asyncio
import aiohttp

async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [session.get(url) for url in urls]
        results = []
        for task in asyncio.as_completed(tasks):
            async with (await task) as resp:
                results.append(await resp.json())
        return results

print(asyncio.run(fetch_all(["https://httpbin.org/json", "https://httpbin.org/get"])))
